In [1]:
import pandas as pd
import numpy as np

from scipy.optimize import minimize

In [2]:
df_all = pd.read_csv('00_data/elections_round2_all_v2.csv')
df_all.head()

,district_number,district_gen_numeric,district_general,district_location,1.К-сть одержаних бюлетенів,"2.К-сть виборців, внесених до списку_x","3.К-сть виборців, внесених до витягу для голос. за місцем перебув.",4.К-сть невикор. бюлетенів,"5.К-сть виборців, які отримали бюлетені у приміщ.","6.К-сть виборців, які отримали бюлетені за місцем перебув.",...,ЗеленськийВолодимир_inflow_ТимошенкоЮлія,ЗеленськийВолодимир_inflow_other_cands,ПорошенкоПетро_inflow_others,ПорошенкоПетро_inflow_БойкоЮрій,ПорошенкоПетро_inflow_ВілкулОлександр,ПорошенкоПетро_inflow_ГриценкоАнатолій,ПорошенкоПетро_inflow_ЛяшкоОлег,ПорошенкоПетро_inflow_СмешкоІгор,ПорошенкоПетро_inflow_ТимошенкоЮлія,ПорошенкоПетро_inflow_other_cands
0,50130,11,"Вінницька область, Територіальний виборчий окр...","Школа-ліцей, спортивна зала",1696,1694,103.0,602,1004,90,...,117.521042,34.074148,129.0,8.272545,2.843687,18.871743,11.633267,31.022044,43.689379,12.667335
1,50131,11,"Вінницька область, Територіальний виборчий окр...","ДНЗ ""Казка"", спортивна зала",2128,2095,15.0,822,1295,11,...,140.054795,48.657534,162.0,11.589041,8.136986,23.917808,11.342466,36.246575,52.520548,18.246575
2,50132,11,"Вінницька область, Територіальний виборчий окр...","ПСП ""Агрофірма Батьківщина"", приіщення контори",1296,1281,4.0,484,808,4,...,74.846361,27.347709,80.0,8.840970,2.156334,10.350404,7.547170,20.485175,22.425876,8.194070
3,50133,11,"Вінницька область, Територіальний виборчий окр...","школа, спортзал",2448,2422,14.0,933,1504,11,...,140.350474,50.428958,185.0,13.768606,7.259811,23.531800,15.270636,46.562923,57.828146,20.778078
4,50134,11,"Вінницька область, Територіальний виборчий окр...","звіргосподарство, кабінет головного бухгалтера",240,222,0.0,108,132,0,...,16.400000,5.046154,16.0,0.246154,0.246154,2.953846,0.738462,3.446154,6.400000,1.969231


In [6]:
candidates = ['ЗеленськийВолодимир','ПорошенкоПетро']

main_cands = ['БойкоЮрій',
             'ВілкулОлександр',
             'ГриценкоАнатолій',
             'ЛяшкоОлег',
             'СмешкоІгор',
             'ТимошенкоЮлія',
             'other_cands']

cands_inflow = [cand+'_inflow' for cand in main_cands]

for candidate in main_cands:
    df_all[candidate+'_inflow'] = df_all[candidate] - df_all[candidate+'_stayed_home']

In [10]:
#source: https://stackoverflow.com/questions/52589230/python-simple-linear-regression-with-constraint

string = '; '

X = df_all[cands_inflow]
print(X.shape)

# Define the Model
model = lambda b, X: b[0] * X.iloc[:,0] + b[1] * X.iloc[:,1] + b[2] * X.iloc[:,2] + b[3] * X.iloc[:,3] + b[4] * X.iloc[:,4] + b[5] * X.iloc[:,5] + b[6] * X.iloc[:,6]

# The objective Function to minimize (least-squares regression)
obj = lambda b, Y, X: np.sum(np.abs(Y-model(b, X))**2)

# Bounds: b[0], b[1], b[2] .... b[6] >= 0
bnds = [(0, 1) for i in cands_inflow]

# Initial guess for b[1], b[2], b[3] .... b[6]:
xinit = np.zeros(len(cands_inflow))

for candidate in candidates:

    Y = df_all[candidate+'_inflow_others']
    res = minimize(obj, args=(Y, X), x0=xinit, bounds=bnds)
    
    print(candidate + ": " + string.join([main_cands[i] + " " + str(res.x[i]) for i in range(len(main_cands))]))

(29608, 7)
ЗеленськийВолодимир: БойкоЮрій 1.0; ВілкулОлександр 1.0; ГриценкоАнатолій 0.0; ЛяшкоОлег 1.0; СмешкоІгор 1.0; ТимошенкоЮлія 1.0; other_cands 0.5865866050335818
ПорошенкоПетро: БойкоЮрій 0.0; ВілкулОлександр 0.0; ГриценкоАнатолій 1.0; ЛяшкоОлег 0.0; СмешкоІгор 0.0; ТимошенкоЮлія 0.0; other_cands 0.4135163363468822
